In [1]:
from openai import OpenAI
import os

# [공지]
현재 하나의 Account / Organization을 여러 사용자가 사용하고 있습니다.    
assistant, vector_store 생성 시 **이름 뒤에 자신을 식별할 수 있는 id를 추가해주세요.** (예. "My Expense 01")

# Assistant API Basic (v2)
- assistant, thread, message, run을 생성할 수 있다. 
- run을 실행하고 결과 message를 출력할 수 있다.

In [2]:
# helper functions
import json

def show_json(obj):
    """객체의 정보를 보여줌"""
    display(json.loads(obj.model_dump_json()))

def pretty_print(messages):
    """메시지를 'role : text' 형태로 출력"""
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


In [3]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

In [5]:
# create assistant
# Assistant 설명: 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님
assistant = client.beta.assistants.create(
    name="assistant ask", # 자신의 식별자 (id)를 이름 뒤에 붙여 주세요. 
    instructions="너는 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님", # system message
    model="gpt-4-turbo", # gpt-4-turbo
)
show_json(assistant)

{'id': 'asst_ai2uVPqsYq3IEtSCUr2xUhgj',
 'created_at': 1713931592,
 'description': None,
 'instructions': '너는 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님',
 'metadata': {},
 'model': 'gpt-4-turbo',
 'name': 'assistant ask',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

In [6]:
# create empty threads
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76',
 'created_at': 1713931647,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [8]:
# create messages to thread
message = client.beta.threads.messages.create(
    thread_id= thread.id, # 위에서 생성한 thread의 id 입력
    role= 'user', # message의 role
    content= 'AI 모델이 너무 크기가 클 때 어떻게 해야 하지?' # message의 content
)
show_json(message)

{'id': 'msg_Ho5IGpAouyxg9CzNEk4H2aBc',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'AI 모델이 너무 크기가 클 때 어떻게 해야 하지?'},
   'type': 'text'}],
 'created_at': 1713931962,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76'}

In [9]:
# create run (thread와 assistant를 연결)
run = client.beta.threads.runs.create(
    thread_id=thread.id, # 위에서 생성한 thread의 id 입력
    assistant_id=assistant.id # 위에서 생성한 assistant의 id 입력
)
show_json(run)

{'id': 'run_EoXKYeG94FN80GqzXmEHJARo',
 'assistant_id': 'asst_ai2uVPqsYq3IEtSCUr2xUhgj',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1713932052,
 'expires_at': 1713932652,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4-turbo',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

In [10]:
# retrieve run to check it's status
import time
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [11]:
run = wait_on_run(run, thread)

In [12]:
show_json(run)

{'id': 'run_EoXKYeG94FN80GqzXmEHJARo',
 'assistant_id': 'asst_ai2uVPqsYq3IEtSCUr2xUhgj',
 'cancelled_at': None,
 'completed_at': 1713932068,
 'created_at': 1713932052,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4-turbo',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1713932053,
 'status': 'completed',
 'thread_id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 429,
  'prompt_tokens': 135,
  'total_tokens': 564},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

In [13]:
# List messages in the thread 
messages = client.beta.threads.messages.list(
    thread_id = thread.id, # 위에서 생성한 thread의 id 입력, 
    order = "asc"
)
show_json(messages)

{'data': [{'id': 'msg_mAfqizvLU1B2HiisBFFlvhpf',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'LLM을 설명할 때 RAG를 같이 설명하는데 RAG에 대해서 설명해줘.'},
     'type': 'text'}],
   'created_at': 1713931781,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76'},
  {'id': 'msg_Ho5IGpAouyxg9CzNEk4H2aBc',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'AI 모델이 너무 크기가 클 때 어떻게 해야 하지?'},
     'type': 'text'}],
   'created_at': 1713931962,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_oQnHTnC9XPRE8wSF4zHU1O76'},
  {'id': 'msg_MU9KiAuGugkSXvdShmhoI

In [14]:
def add_message_run(assistant_id:str, thread_id:str, user_message:str):
    """
    1. thread에 새로운 메시지를 추가하고 
    2. run을 생성해 어시스턴트를 실행
    """
    # 1. thread에 새로운 메시지를 추가하고 
    message = client.beta.threads.messages.create(
        thread_id= thread_id, # 위에서 생성한 thread의 id 입력
        role= 'user', # message의 role
        content= user_message # message의 content
    )
    # 2. run을 생성해 어시스턴트를 실행
    run = client.beta.threads.runs.create(
        thread_id=thread_id, # 위에서 생성한 thread의 id 입력
        assistant_id=assistant_id # 위에서 생성한 assistant의 id 입력
    )
    
    return run


def get_response(thread_id):
    """
    Thread의 메시지를 나열
    """
    # Thread의 메시지를 나열 (생성일 기준 오름차순)
    messages = client.beta.threads.messages.list(
        thread_id = thread_id, # 위에서 생성한 thread의 id 입력, 
        order = "asc"
    )
    return messages

In [16]:
# test
TECH_ASSISTANT_ID = assistant.id
CUR_THREAD_ID = thread.id

# user_message = "음... 그럼 모델 프루닝이 무엇이고 어떻게 하는 거야?" # 새로운 메시지 입력
user_message = "그러면 그 방법을 사용해서 모델을 가볍게 만들면 뭐가 좋아?" # 새로운 메시지 입력
run = add_message_run(TECH_ASSISTANT_ID, CUR_THREAD_ID, user_message)
run = wait_on_run(run, thread) 
messages = get_response(CUR_THREAD_ID)
pretty_print(messages)

# Messages
user: LLM을 설명할 때 RAG를 같이 설명하는데 RAG에 대해서 설명해줘.
user: AI 모델이 너무 크기가 클 때 어떻게 해야 하지?
assistant: AI 모델이 너무 클 때는 마치 대형 트럭이 좁은 길을 지나가야 할 때처럼 어려움을 겪을 수 있습니다. 이 문제를 해결하는 방법 중 하나는 모델의 크기를 줄이는 것입니다. 이것을 모델 프루닝(model pruning)이라고 부르는데, 불필요한 부분을 절삭하여 더 작고 효율적인 모델을 만드는 과정입니다.

또 다른 방법은 모델 파티셔닝(model partitioning)입니다. 이는 큰 모델을 여러 작은 조각으로 나누어 각각의 조각을 다른 서버나 프로세서에서 동시에 처리할 수 있도록 하는 기법이죠. 마치 큰 가구를 여러 조각으로 분해하여 좁은 문을 통해 이동시키는 것과 비슷합니다.

마지막으로, 모델 양자화(model quantization)를 사용할 수 있습니다. 이는 모델이 사용하는 데이터의 정밀도를 낮추어 계산량과 메모리 사용량을 줄이는 기술입니다. 이 방법은 마치 고화질의 사진을 적절한 해상도로 줄여서 저장공간을 절약하는 것과 비슷하게 작동합니다.

이러한 방법들을 통해 AI 모델의 크기를 관리하고, 효율적으로 운영할 수 있습니다.
user: 음... 그럼 모델 프로닝이 무엇이고 어떻게 하는 거야?
assistant: 모델 프루닝(Model Pruning)이란, 인공지능 모델에서 중요도가 낮은 파라미터나 연결을 제거함으로써 모델의 크기를 줄이고, 처리 속도를 향상시키는 기법을 말해요. 즉, 네트워크 안에서 중요하지 않은 부분을 '가지치기'하는 것입니다.

모델 프루닝을 하는 방법을 쉽게 설명드리자면, 마치 정원에서 건강하지 않거나 불필요한 가지를 잘라내 건강한 나무를 유지하는 것과 비슷합니다. 구체적으로는 다음과 같은 과정을 포함합니다:

1. **중요도 평가**: 먼저, 모델의 각 가중치(weight)의 중요도를 평가합니다. 이는 그 가중치가 모델의 출력에 얼마나 큰 영향

# Assistant API Advanced
- Assistant를 업데이트 할 수 있다.
    - `VectorStore` 생성
    - `tools`에 `file_search` 추가
- [추가] annotation을 표시할 수 있다.

In [17]:
# Upload the files (SKIP)
file = client.files.create(
    file=open(
        "./data/coffeechat1.txt",
        "rb",
    ),
    purpose="assistants",
)

file = client.files.create(
    file=open(
        "./data/coffeechat2.txt",
        "rb",
    ),
    purpose="assistants",
)

In [18]:
client.files.list().data

[FileObject(id='file-StNE32pHiVzosq3eY9PbYjfw', bytes=11217, created_at=1713933656, filename='coffeechat2.txt', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-IkfTcOBo6mU1oVbGqzaLMjUt', bytes=18742, created_at=1713933655, filename='coffeechat1.txt', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-dWqc4HtyKMfneijZaGtc3A9B', bytes=11217, created_at=1713933635, filename='coffeechat2.txt', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-TNpKiB4sCig4yg9MAAOes0ym', bytes=18742, created_at=1713933634, filename='coffeechat1.txt', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-jRGtDpy9tHBFQS6r2nsoyAGr', bytes=11217, created_at=1713933629, filename='coffeechat2.txt', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-wetyz5mML28Bmxp7sT0RCXyH

In [19]:
# view files
for file_obj in client.files.list(purpose="assistants").data:
    show_json(file_obj)

{'id': 'file-StNE32pHiVzosq3eY9PbYjfw',
 'bytes': 11217,
 'created_at': 1713933656,
 'filename': 'coffeechat2.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-IkfTcOBo6mU1oVbGqzaLMjUt',
 'bytes': 18742,
 'created_at': 1713933655,
 'filename': 'coffeechat1.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-dWqc4HtyKMfneijZaGtc3A9B',
 'bytes': 11217,
 'created_at': 1713933635,
 'filename': 'coffeechat2.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-TNpKiB4sCig4yg9MAAOes0ym',
 'bytes': 18742,
 'created_at': 1713933634,
 'filename': 'coffeechat1.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-jRGtDpy9tHBFQS6r2nsoyAGr',
 'bytes': 11217,
 'created_at': 1713933629,
 'filename': 'coffeechat2.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

...

{'id': 'file-OpvPi0WPe9fFL48IX4xjLDQU',
 'bytes': 18742,
 'created_at': 1713933593,
 'filename': 'coffeechat1.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-w4jY5bTblb8GgJEzCC58QpxE',
 'bytes': 11217,
 'created_at': 1713933540,
 'filename': 'coffeechat2.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-xVxsyneVPP5G8wBjG8Fp2GfR',
 'bytes': 18742,
 'created_at': 1713933539,
 'filename': 'coffeechat1.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-NPT59yxwKHWWvMYFpBEMg8lh',
 'bytes': 11217,
 'created_at': 1713933235,
 'filename': 'coffeechat2.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

{'id': 'file-ZvSBuyKcICYfMYlMV1PeAiJj',
 'bytes': 18742,
 'created_at': 1713933234,
 'filename': 'coffeechat1.txt',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

In [20]:
# vector store 생성
vector_store = client.beta.vector_stores.create(
  name="Quantization Coffee Chat - ahnsk",
  #file_ids=[file_obj.id for file_obj in client.files.list(purpose="assistants").data],
  file_ids=["file-ZvSBuyKcICYfMYlMV1PeAiJj", "file-NPT59yxwKHWWvMYFpBEMg8lh"], # coffeechat1.txt 과 coffeechat2.txt 
)
show_json(vector_store)

{'id': 'vs_ZjiJs9RLtMp6eVifE4tZRoc6',
 'bytes': None,
 'created_at': 1713933977,
 'file_counts': {'cancelled': 0,
  'completed': 2,
  'failed': 0,
  'in_progress': 0,
  'total': 2},
 'last_active_at': 1713933977,
 'metadata': {},
 'name': 'Quantization Coffee Chat - ahnsk',
 'object': 'vector_store',
 'status': 'completed',
 'expires_after': None,
 'expires_at': None,
 'usage_bytes': 37451}

In [31]:
# Update Assistant: file_search 기능을 추가
assistant = client.beta.assistants.update(
    assistant_id = TECH_ASSISTANT_ID, # assistant ID
    tools = [{"type": "file_search"}], # file_search 기능을 추가
    tool_resources = {"file_search": {
            "vector_store_ids": [vector_store.id]
        }
    } # tools_resource에 vectorDB를 추가
)
show_json(assistant)

{'id': 'asst_ai2uVPqsYq3IEtSCUr2xUhgj',
 'created_at': 1713931592,
 'description': None,
 'instructions': '너는 복잡하고 어려운 IT 또는 컴퓨터 공학 지식을 비유를 사용해서 알기 쉽게 그리고 간결하게 설명해주는 나만의 IT 선생님',
 'metadata': {},
 'model': 'gpt-4-turbo',
 'name': 'assistant ask',
 'object': 'assistant',
 'tools': [{'type': 'file_search'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': ['vs_ZjiJs9RLtMp6eVifE4tZRoc6']}},
 'top_p': 1.0}

In [32]:
# test
# user_message = "인터뷰에서 가장 중요하게 다룬 경량화 종류는 무엇이야?" # 새로운 메시지 입력 ("인터뷰에서 가장 중요하게 다룬 경량화 종류는 무엇이야?")
user_message = "이번 인터뷰에 어떤 사람이 나와서 무슨 이야기해?"
run = add_message_run(TECH_ASSISTANT_ID, CUR_THREAD_ID, user_message) # 메시지를 보내고 run을 생성
run = wait_on_run(run, thread) # run 완료까지 대기
messages = get_response(CUR_THREAD_ID) # 결과 확인
pretty_print(messages)

# Messages
user: LLM을 설명할 때 RAG를 같이 설명하는데 RAG에 대해서 설명해줘.
user: AI 모델이 너무 크기가 클 때 어떻게 해야 하지?
assistant: AI 모델이 너무 클 때는 마치 대형 트럭이 좁은 길을 지나가야 할 때처럼 어려움을 겪을 수 있습니다. 이 문제를 해결하는 방법 중 하나는 모델의 크기를 줄이는 것입니다. 이것을 모델 프루닝(model pruning)이라고 부르는데, 불필요한 부분을 절삭하여 더 작고 효율적인 모델을 만드는 과정입니다.

또 다른 방법은 모델 파티셔닝(model partitioning)입니다. 이는 큰 모델을 여러 작은 조각으로 나누어 각각의 조각을 다른 서버나 프로세서에서 동시에 처리할 수 있도록 하는 기법이죠. 마치 큰 가구를 여러 조각으로 분해하여 좁은 문을 통해 이동시키는 것과 비슷합니다.

마지막으로, 모델 양자화(model quantization)를 사용할 수 있습니다. 이는 모델이 사용하는 데이터의 정밀도를 낮추어 계산량과 메모리 사용량을 줄이는 기술입니다. 이 방법은 마치 고화질의 사진을 적절한 해상도로 줄여서 저장공간을 절약하는 것과 비슷하게 작동합니다.

이러한 방법들을 통해 AI 모델의 크기를 관리하고, 효율적으로 운영할 수 있습니다.
user: 음... 그럼 모델 프로닝이 무엇이고 어떻게 하는 거야?
assistant: 모델 프루닝(Model Pruning)이란, 인공지능 모델에서 중요도가 낮은 파라미터나 연결을 제거함으로써 모델의 크기를 줄이고, 처리 속도를 향상시키는 기법을 말해요. 즉, 네트워크 안에서 중요하지 않은 부분을 '가지치기'하는 것입니다.

모델 프루닝을 하는 방법을 쉽게 설명드리자면, 마치 정원에서 건강하지 않거나 불필요한 가지를 잘라내 건강한 나무를 유지하는 것과 비슷합니다. 구체적으로는 다음과 같은 과정을 포함합니다:

1. **중요도 평가**: 먼저, 모델의 각 가중치(weight)의 중요도를 평가합니다. 이는 그 가중치가 모델의 출력에 얼마나 큰 영향

In [36]:
messages.data[-1].content[0]

TextContentBlock(text=Text(annotations=[], value='이번 인터뷰에는 김태수 씨가 나와서 주로 모델 경량화에 대해 이야기했습니다. 그는 KAIST와 포항공과대학교(POSTECH)에서 공부하고 현재 SqueezeBits라는 회사에서 일하고 있으며, 그의 전문 분야는 AI 모델의 경량화입니다. 인터뷰에서 그는 모델 경량화의 중요성을 강조하며 이 기술이 어떻게 AI 모델을 더 작고 효율적으로 만드는지 설명했습니다. 주요 논의 주제는 Pruning과 Knowledge Distillation이었으며, 이 기술들을 통해 고성능 모델의 지식을 더 작은 모델에 전달하는 방법에 대해 토론되었습니다 .'), type='text')

In [34]:
# [참고] Retrieval의 경우 annotation (어떤 파일을 참고했는지) 정보를 확인할 수 있음
messages.data[-1].content[0].text

Text(annotations=[], value='이번 인터뷰에는 김태수 씨가 나와서 주로 모델 경량화에 대해 이야기했습니다. 그는 KAIST와 포항공과대학교(POSTECH)에서 공부하고 현재 SqueezeBits라는 회사에서 일하고 있으며, 그의 전문 분야는 AI 모델의 경량화입니다. 인터뷰에서 그는 모델 경량화의 중요성을 강조하며 이 기술이 어떻게 AI 모델을 더 작고 효율적으로 만드는지 설명했습니다. 주요 논의 주제는 Pruning과 Knowledge Distillation이었으며, 이 기술들을 통해 고성능 모델의 지식을 더 작은 모델에 전달하는 방법에 대해 토론되었습니다 .')

In [37]:
# [참고] annotation의 활용: footprint(ex. [0], [1], ...)로 표현하기
message_content = messages.data[-1].content[0].text
annotations = messages.data[-1].content[0].text.annotations
citations = []
for index, annotation in enumerate(annotations):
    # 텍스트에서 annotation을 footnote로 대체
    message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

    if getattr(annotation, 'file_citation', None):
        file_citation = getattr(annotation, 'file_citation', None)
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)

In [38]:
pretty_print(messages)

# Messages
user: LLM을 설명할 때 RAG를 같이 설명하는데 RAG에 대해서 설명해줘.
user: AI 모델이 너무 크기가 클 때 어떻게 해야 하지?
assistant: AI 모델이 너무 클 때는 마치 대형 트럭이 좁은 길을 지나가야 할 때처럼 어려움을 겪을 수 있습니다. 이 문제를 해결하는 방법 중 하나는 모델의 크기를 줄이는 것입니다. 이것을 모델 프루닝(model pruning)이라고 부르는데, 불필요한 부분을 절삭하여 더 작고 효율적인 모델을 만드는 과정입니다.

또 다른 방법은 모델 파티셔닝(model partitioning)입니다. 이는 큰 모델을 여러 작은 조각으로 나누어 각각의 조각을 다른 서버나 프로세서에서 동시에 처리할 수 있도록 하는 기법이죠. 마치 큰 가구를 여러 조각으로 분해하여 좁은 문을 통해 이동시키는 것과 비슷합니다.

마지막으로, 모델 양자화(model quantization)를 사용할 수 있습니다. 이는 모델이 사용하는 데이터의 정밀도를 낮추어 계산량과 메모리 사용량을 줄이는 기술입니다. 이 방법은 마치 고화질의 사진을 적절한 해상도로 줄여서 저장공간을 절약하는 것과 비슷하게 작동합니다.

이러한 방법들을 통해 AI 모델의 크기를 관리하고, 효율적으로 운영할 수 있습니다.
user: 음... 그럼 모델 프로닝이 무엇이고 어떻게 하는 거야?
assistant: 모델 프루닝(Model Pruning)이란, 인공지능 모델에서 중요도가 낮은 파라미터나 연결을 제거함으로써 모델의 크기를 줄이고, 처리 속도를 향상시키는 기법을 말해요. 즉, 네트워크 안에서 중요하지 않은 부분을 '가지치기'하는 것입니다.

모델 프루닝을 하는 방법을 쉽게 설명드리자면, 마치 정원에서 건강하지 않거나 불필요한 가지를 잘라내 건강한 나무를 유지하는 것과 비슷합니다. 구체적으로는 다음과 같은 과정을 포함합니다:

1. **중요도 평가**: 먼저, 모델의 각 가중치(weight)의 중요도를 평가합니다. 이는 그 가중치가 모델의 출력에 얼마나 큰 영향

# [도전] 나만의 Finance Assistant 생성
- Finance Assistant 설명
    - 나의 지출 데이터를 보고 지출 상태를 시각화 해줍니다. (그래프 생성)
    - 나의 지출 데이터 기반으로 전문적인 지출 관리 조언을 제공해줍니다.
- Assistant의 tools에 "file_search"와 "code_interpreter" 기능을 추가합니다.
- "expenses.md" 파일을 업로드해 vectore store를 생성하고 이를 tools_resource에 입력합니다.
- "나의 지출 상태를 바 차트로 시각화 해줘"라는 메시지를 보내고 응답을 받습니다.